# MCP Setup and Integration with NVIDIA NeMo Agent Toolkit

In this notebook we will walk through client and server support for [Model Context Protocol (MCP)](https://modelcontextprotocol.io/) in the NeMo Agent toolkit.

Detailed documentation can be found in the [MCP Docs](../../docs/source/workflows/mcp/index.md), which inspired the execution flow of this notebook.

**Goal**

By the end of this notebook, you will be able to:
- Use agents as an **MCP server** to publish tools using MCP to be used by any MCP client.
- Use agents as an **MCP client** to connect to and use tools served by remote MCP servers.


## Table of Contents

- [0.0) Setup](#setup)
  - [0.1) Prerequisites](#prereqs)
  - [0.2) API keys](#api-keys)
  - [0.3) Installing NeMo Agent Toolkit](#install-nat)
  - [0.4) Additional dependencies](#deps)
- [1.0) NeMo Agent Toolkit as an MCP Server](#mcp-servers)
  - [1.1) Starting the server with default settings](#start-mcp-servers)
  - [1.2) Starting the Server with optional flags](#start-mcp-servers-flags)
  - [1.3) Filtering tools available from the server](#start-mcp-server-filter-tools)
  - [1.4) Options for listing available tools](#server-listing-tool-options)
- [2.0) NeMo Agent Toolkit as an MCP client](#start-mcp-client)
  - [2.1) Configure and run MCP clients using function groups](#mcp-client-function-groups)
  - [2.2) Listing tools from MCP servers](#mcp-client-tool-calls)
  - [2.3) Inspecting MCP client integration in workflows](#mcp-client-tools)
  - [2.4) Calling individual MCP tools](#client-tools)
- [3.0) Next steps](#next-steps)

<a id="setup"></a>
# 0.0) Setup

<a id="prereqs"></a>
## 0.1) Prerequisites

We strongly recommend that users begin this notebook with a working understanding of NeMo Agent toolkit workflows. Please refer to earlier iterations of this notebook series prior to beginning this notebook.

- **Platform:** Linux, macOS, or Windows
- **Python:** version 3.11, 3.12, or 3.13
- **Python Packages:** `pip`

<a id="api-keys"></a>
## 0.2) API Keys

For this notebook, you will need the following API keys to run all examples end-to-end:

- **NVIDIA Build:** You can obtain an NVIDIA Build API Key by creating an [NVIDIA Build](https://build.nvidia.com) account and generating a key at https://build.nvidia.com/settings/api-keys

Then you can run the cell below:

In [ ]:
import getpass
import os

if "NVIDIA_API_KEY" not in os.environ:
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key

<a id="install-nat"></a>
## 0.3) Installing NeMo Agent Toolkit

NeMo Agent toolkit can be installed through the PyPI `nvidia-nat` package, the recommended way to install `nvidia-nat` is through `uv`.

First, we will install `uv` which offers parallel downloads and faster dependency resolution.

In [ ]:
%%bash
pip install uv

There are several optional subpackages available for `nvidia-nat`. For this example, we will rely on two subpackages:
* The `nvidia-nat[langchain]` subpackage contains components for integrating with [LangChain](https://python.langchain.com/docs/introduction/).
* The `nvidia-nat[mcp]` subpackage contains components for MCP clients and servers with NeMo Agent toolkit.

In [ ]:
%%bash
uv pip show -q "nvidia-nat-langchain"
nat_langchain_installed=$?
uv pip show -q "nvidia-nat-mcp"
nat_mcp_installed=$?
if [[ ${nat_langchain_installed} -ne 0 || ${nat_mcp_installed} -ne 0 ]]; then
    uv pip install "nvidia-nat[langchain,mcp]"
else
    echo "nvidia-nat[langchain,mcp] is already installed"
fi

<a id="deps"></a>
## 0.4) Additional dependencies

In [ ]:
%%bash
uv pip show -q "mcp-server-time"
mcp_time_installed=$?
if [[ ${mcp_time_installed} -ne 0 ]]; then
    uv pip install "mcp-server-time"
else
    echo "mcp-server-time is already installed"
fi

In [ ]:
# install tomli-w to edit pyproject.toml and rename the package
!uv pip install tomli tomli-w

Next we will define a utility that will help us source the correct example code depending on your runtime environment and branch.

In [ ]:
def install_example_code(example_code_path: str):
    """
    This function prompts the user to select an installation option for the example code.

    Args:
        example_code_path (str): The path to the example code to install.
    """
    import shutil
    import subprocess
    from pathlib import Path

    def rename_package_in_pyproject(pyproject_path: Path, suffix: str = "_notebook") -> str:
        """
        Rename the package in pyproject.toml to avoid conflicts with existing installations.

        Args:
            pyproject_path: Path to the pyproject.toml file
            suffix: Suffix to append to the package name

        Returns:
            The new package name, or None if renaming failed
        """
        try:
            # Try Python 3.11+ built-in tomllib first
            try:
                import tomllib as tomli
            except ImportError:
                import tomli
            import tomli_w
        except ImportError:
            print("Warning: Could not import toml library, package name will not be modified")
            return None

        try:
            # Read the current pyproject.toml
            with open(pyproject_path, 'rb') as f:
                config = tomli.load(f)

            # Get the original package name
            original_name = config.get('project', {}).get('name')
            if not original_name:
                print("Warning: No package name found in pyproject.toml")
                return None

            # Create new unique name
            new_name = f"{original_name}{suffix}"

            # Update the package name
            config['project']['name'] = new_name

            # Write back to pyproject.toml
            with open(pyproject_path, 'wb') as f:
                tomli_w.dump(config, f)

            print(f"✓ Renamed package from '{original_name}' to '{new_name}' to avoid conflicts")
            return new_name

        except Exception as e:
            print(f"Warning: Could not modify package name: {e}")
            return None

    # Detect if running in Colab
    try:
        import google.colab  # type: ignore  # noqa: F401
    except ImportError:
        in_colab = False
        base_path = Path(os.getcwd())
    else:
        in_colab = True
        base_path = Path("/content")

    # Simple input prompt for branch selection
    print("=" * 60)
    print("Example code installation for: ", example_code_path)
    print("=" * 60)
    print("\nOptions:")
    if not in_colab:
        print("  - Enter 'local' to copy example code to current directory")
    print("  - Enter a branch name (e.g., 'develop', 'main') to clone and install")
    print("=" * 60)

    from IPython.core.error import StdinNotImplementedError
    try:
        branch_name = input("\nEnter your choice: ").strip()
    except StdinNotImplementedError:
        branch_name = 'local'
        print(f"\nNo input available. Defaulting to branch: {branch_name}")

    # Default to 'main' if empty
    if not branch_name:
        branch_name = 'main'

    if branch_name.lower() == 'local' and not in_colab:
        # Copy files from local repository to current directory
        print("\nCopying example code from local repository...")

        # Try to find the repository root
        current = Path.cwd()
        repo_root = None

        # Look for repository root by searching for pyproject.toml
        for parent in [current] + list(current.parents):
            if (parent / "pyproject.toml").exists() and (parent / "src" / "nat").exists():
                repo_root = parent
                break

        if repo_root is None:
            print("✗ Error: Could not find repository root.")
            print("Please use git install instead.")
            return

        # path-check-skip-next-line
        source_path = repo_root / example_code_path

        # Get the example folder name (e.g., "simple_calculator")
        example_name = Path(example_code_path).name
        dest_path = base_path / f"{example_name}_notebook"

        if source_path.exists():
            # Remove destination if it already exists
            if dest_path.exists():
                print(f"Removing existing directory: {dest_path}")
                shutil.rmtree(dest_path)

            # Copy the entire example directory
            shutil.copytree(source_path, dest_path)
            print(f"✓ Copied example code to: {dest_path.absolute()}")

            # Optional: Install dependencies if there's a pyproject.toml
            pyproject_file = dest_path / "pyproject.toml"
            if pyproject_file.exists():
                # Rename package to avoid conflicts
                new_name = rename_package_in_pyproject(pyproject_file)

                print("\nInstalling example package in editable mode...")
                get_ipython().system(f'pip install --no-deps -e {dest_path}')

                if new_name:
                    print(f"✓ Installed package as '{new_name}' from: {dest_path}")
                else:
                    print(f"✓ Installed package from: {dest_path}")

            print(f"\nYou can now edit the files in: {dest_path}")
        else:
            print(f"✗ Error: Source path not found: {source_path.absolute()}")
            print("Make sure you're running this from the correct directory")

    elif branch_name.lower() == 'local' and in_colab:
        print("\n✗ Local install not available in Google Colab.")
        print("Please specify a branch name to clone from git.")

    else:
        # Clone full repo and install from there
        if in_colab:
            print(f"\nCloning branch '{branch_name}' from GitHub...")
            repo_path = base_path / "NeMo-Agent-Toolkit"
        else:
            print(f"\nInstalling example code from branch: {branch_name}")
            repo_path = base_path / f"NeMo-Agent-Toolkit-{branch_name}"

        repo_url = "https://github.com/NVIDIA/NeMo-Agent-Toolkit.git"

        try:
            # Remove existing repo if it exists
            if repo_path.exists():
                print(f"Removing existing repository at: {repo_path}")
                shutil.rmtree(repo_path)

            # Clone the full repository
            print(f"Cloning repository (branch: {branch_name})...")
            _ = subprocess.run(
                ["git", "clone", "--branch", branch_name, repo_url, str(repo_path)],
                capture_output=True,
                text=True,
                check=True
            )
            print(f"✓ Cloned repository to: {repo_path}")

            # Verify the example path exists
            example_path = repo_path / example_code_path
            if not example_path.exists():
                print(f"✗ Error: Path '{example_code_path}' not found in branch '{branch_name}'")
                return

            # Install the example package
            pyproject_file = example_path / "pyproject.toml"
            if pyproject_file.exists():
                # Rename package to avoid conflicts
                new_name = rename_package_in_pyproject(pyproject_file)

                print("\nInstalling example package in editable mode...")
                print(f"Installing from: {example_path}")
                # get_ipython().system(f'pip install --no-deps -e {example_path}')
                get_ipython().system(f'pip install --no-deps {example_path}')

                if new_name:
                    print(f"✓ Installed package as '{new_name}'")
                else:
                    print("✓ Installed package")

            # Get the example folder name
            example_name = Path(example_code_path).name

            print("\n✓ Setup complete!")
            print(f"Repository location: {repo_path}")
            print(f"Example config: {example_path / 'configs'}")
            print("\nYou can reference configs using:")
            print(f"  {example_path / 'configs/config.yml'}")

            # Optionally copy configs to working directory for easy access
            dest_path = base_path / f"{example_name}_notebook"
            if dest_path.exists():
                shutil.rmtree(dest_path)
            shutil.copytree(example_path, dest_path)
            print(f"\nAlso copied example to: {dest_path}")
            print(f"Use configs from: {dest_path / 'configs/config.yml'}")

            # Remove existing repo if it exists
            if repo_path.exists():
                print(f"Removing existing repository at: {repo_path}")
                shutil.rmtree(repo_path)


        except subprocess.CalledProcessError as e:
            print(f"✗ Error cloning repository: {e.stderr}")
            print(f"Make sure branch '{branch_name}' exists.")
        except Exception as e:
            print(f"✗ Error: {e}")

    print("\n" + "=" * 60)

<a id="mcp-servers"></a>
# 1.0) NeMo Agent Toolkit as an MCP Server

MCP uses a `client-server` architecture, where MCP clients connect to servers using one of a few transport types to discover and invoke tool calls from the distributed MCP servers. This core concept in agentic AI enables agents to determine which servers to invoke given the intent of the input prompt, and delegate tasks to their most appropriate service.

Therefore, it's logical to start the tutorial of NeMo Agent toolkit support for MCP from the MCP server, where the new tools are hosted and called.

The `nat mcp serve` command line utility can start an MCP server that publishes the functions from your workflow as MCP tools. In this section we are going to start up the `simple_calculator` server that we used to demonstrate the client-to-server connection in the first section, and dive deeper into hosting MCP servers with the toolkit.

<a id="start-mcp-servers"></a>
## 1.1) Starting the server with default settings

First, let's install the `simple_calculator` example from source as `simple_calculator_notebook`, which we can edit and run.

We recommend using the 'develop' branch in Google Colab.

In [ ]:
install_example_code("examples/getting_started/simple_calculator")

In [ ]:
%%bash --bg
# Then let's start up the simple calculator MCP server.
nat mcp serve --config_file ./simple_calculator_notebook/configs/config.yml

The `--bg` command will run your server in the background of the notebook, so you might not see confirmation that the server is running. To see the startup logs, you can remove this flag and run the cell again. However, subsequent steps require the server to be run as a background process.

Next, we will list the set of available tools from `simple_calculator_notebook` MCP server.

In [ ]:
# Wait about 10 seconds before running the next cell...
import time

time.sleep(10)

In [ ]:
!nat mcp client tool list --url http://localhost:9901/mcp

The steps above have loaded the workflow configuration from the specified file, started an MCP server on the default host (localhost) and port (9901), and published all tools from the workflow as MCP tools. The MCP server is available at `http://localhost:9901/mcp` using streamable-http transport.

You can also use the `sse` (Server-Sent Events) transport for backwards compatibility through the `--transport` flag, for example:
```bash
nat mcp serve --config_file ./simple_calculator_notebook/configs/config.yml --transport sse
```
With this configuration, the MCP server is available at `http://localhost:9901/sse` using SSE transport.

<div style="color: red; font-style: italic;">
<strong>WARNING - SSE Transport Security Limitations</strong>: The SSE transport does not support authentication. For production deployments, use `streamable-http` transport with authentication configured. SSE should only be used for local development on localhost or behind an authenticating reverse proxy.
</div>

<a id="start-mcp-servers-flags"></a>
## 1.2) Starting the server with optional flags

You can optionally specify the optional server runtime settings (host, port, process name) using the following flags, without directly editing the configuration file.

In [ ]:
# let's kill anything using port 9901 as it may be running from a previous section
!lsof -ti:9901 | xargs kill -9

In [ ]:
%%bash --bg
nat mcp serve --config_file ./simple_calculator_notebook/configs/config.yml \
  --host 0.0.0.0 \
  --port 9901 \
  --name "My MCP Server"

In [ ]:
# Wait about 10 seconds before running the next cell...
import time

time.sleep(10)

In [ ]:
# For streamable-http transport (default)
!nat mcp client tool list --url http://localhost:9901/mcp

<a id="start-mcp-server-filter-tools"></a>
## 1.3) Filtering tools available from the server

The `--tool_names` flag allows the user to specify a subset of tools that will be enabled on the server. In this example below, we will only enable `calculator__multiply` and `calculator__divide`

In [ ]:
# let's kill anything using port 9901 as it may be running from a previous section
!lsof -ti:9901 | xargs kill -9

In [ ]:
%%bash --bg
nat mcp serve --config_file ./simple_calculator_notebook/configs/config.yml \
  --tool_names calculator__multiply \
  --tool_names calculator__divide \
  --host 0.0.0.0 \
  --port 9901 \
  --name "My MCP Server"

In [ ]:
# Wait about 10 seconds before running the next cell...
import time

time.sleep(10)

In [ ]:
# For streamable-http transport (default)
!nat mcp client tool list --url http://localhost:9901/mcp

Note in the above output that only `calculator__multiply` and `calculator__divide` are now available from the server.

<a id="server-listing-tool-options"></a>
## 1.4) Options for listing available tools

After running, you can verify the tool list available on that server using the `nat` command line utilities.

In [ ]:
%%bash
nat mcp client tool list

Or by inspecting the exposed endpoint

In [ ]:
%%bash
curl -s http://localhost:9901/debug/tools/list | jq

You can request one or more specific tools by name. The `name` parameter accepts repeated values or a comma‑separated list. When `name` is provided, detailed schemas are returned by default:

In [ ]:
%%bash
# Single tool (detailed by default)
curl -s "http://localhost:9901/debug/tools/list?name=calculator__multiply" | jq

# Multiple tools (detailed by default)
curl -s "http://localhost:9901/debug/tools/list?name=calculator__multiply&name=calculator__divide" | jq

# Comma-separated list (equivalent)
curl -s "http://localhost:9901/debug/tools/list?name=calculator__multiply,calculator__divide" | jq


You can control the amount of detail using the `detail` query parameter:

- When requesting specific tool(s) with `name`, detailed schema is returned by default. Pass `detail=false` to suppress schemas:


In [ ]:
%%bash
curl -s "http://localhost:9901/debug/tools/list?name=calculator__multiply&detail=false" | jq

- When listing all tools (without `name`), the default output is simplified. Pass `detail=true` to include schemas for each tool:


In [ ]:
%%bash
curl -s "http://localhost:9901/debug/tools/list?detail=true" | jq

<a id="start-mcp-client"></a>
# 2.0) NeMo Agent Toolkit as an MCP client

Using NeMo Agent toolkit to connect to MCP servers as a client uses `mcp_client`, a flexible configuration using function groups that allows you to connect to an MCP server, dynamically discover the tools it serves, and register them as NeMo Agent toolkit functions.

In this section we are going to walk through use cases demonstrating this capability.

<a id="mcp-client-function-groups"></a>
## 2.1) Configure and run MCP clients using function groups

You can use the `mcp_client` function group to connect to an MCP server, dynamically discover the tools it serves, and register them as NeMo Agent toolkit functions. 

Key Features:
- **Automatic tool discovery** from the MCP Server by the client.
- **Easy YAML configurations**: The function group can be directly referenced in the workflow configuration. Multiple function groups can be used in the same workflow to access tools from multiple MCP servers. Refer to [Function Groups](../../docs/source/workflows/function-groups.md) for more information about function group capabilities.
    - A tool within a function group can also be referenced by its name using the following syntax: `<function_group_name>.<tool_name>`.
    - The function group supports filtering using the `include` and `exclude` parameters. You can also optionally override the tool name and description defined by the MCP server using the `tool_overrides` parameter. Note that `tool_overrides` require that the function is explicitly listed under the optional `include` list of the function group configuration. See [function group accessibility](../../docs/source/workflows/function-groups.md#understanding-function-accessibility) for more details.


Here is a nice example of function group MCP client usage
```
function_groups:
  mcp_tools:
    _type: mcp_client
    server:
      transport: streamable-http
      url: "http://localhost:9901/mcp"
    include:
      - tool_a
      - tool_b
    tool_overrides:
      tool_a:
        alias: "tool_a_alias"
        description: "Tool A description"

workflow:
  _type: react_agent
  tool_names:
    - mcp_tools
```

The `mcp_client` function group supports filtering using the `include` and `exclude` parameters. You can also optionally override the tool name and description defined by the MCP server using the `tool_overrides` parameter.

If the server from Part 1 is still running, lets stop it.

In [ ]:
# let's kill anything using port 9901 before moving on to the next section for posterity
!lsof -ti:9901 | xargs kill -9

Let's set up an MCP client using function groups.

First we create a workflow with the standard `nat workflow create` command line utility.

In [ ]:
!nat workflow create mcp_dev_workflow

Then let's add a new configuration for this MCP test. This configuration defines two function groups `mcp_time` and `mcp_math`.

`mcp_time`: This `function_group` inherits from `_type: mcp_client`, which by default uses the streamable-http transport. However, we are overwriting this transport in lieu of the stdio transport, which requires us to define a startup command for the MCP server. This is necessary because the `mcp-server-time` package is designed to run as a local Python module rather than as a remote HTTP service. The stdio transport allows NeMo Agent toolkit to launch and communicate with the MCP server directly through standard input/output streams, making it ideal for locally-installed MCP servers. We specify the command `python -m mcp_server_time` to start the server process, along with any required arguments like the timezone configuration.

`mcp_math`: This `function_group` also inherits from `_type: mcp_client`, but unlike `mcp_time`, it uses the default streamable-http transport. This transport type is designed for MCP servers that are already running as HTTP services. We configure it by specifying `url: "http://localhost:9901/mcp"`, which tells the client where to connect to discover and invoke the available mathematical tools. The streamable-http transport is ideal for remote or containerized MCP servers that expose their functionality over HTTP endpoints, allowing for more flexible deployment scenarios compared to the stdio transport. However, this function group does require that the `mcp_math` server is up and running in advance.

In [ ]:
%%writefile ./mcp_dev_workflow/configs/mcp_config.yml
function_groups:
  mcp_time:
    _type: mcp_client
    server:
      transport: stdio
      command: "python"
      args: ["-m", "mcp_server_time", "--local-timezone=America/Los_Angeles"]
  mcp_math:
    _type: mcp_client
    server:
      transport: streamable-http
      url: "http://localhost:9901/mcp"

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    temperature: 0.0
    max_tokens: 250

workflow:
  _type: react_agent
  llm_name: nim_llm
  tool_names:
    - mcp_time
    - mcp_math
  verbose: true

Before running this workflow we need to start the `simple_calculator` server again

In [ ]:
%%bash --bg
nat mcp serve --config_file simple_calculator_notebook/configs/config.yml \
  --host 0.0.0.0 \
  --port 9901 \
  --name "My MCP Server"

In [ ]:
# Wait about 10 seconds before running the next cell...
import time

time.sleep(10)

In [ ]:
!nat run --config_file ./mcp_dev_workflow/configs/mcp_config.yml \
    --input "Is the product of 2 * 4 greater than the current hour of the day in Los Angeles?"

<a id="mcp-client-tool-calls"></a>
## 2.2) Listing tools from MCP servers

Use the `nat mcp client` commands to inspect and call tools available from an MCP server before running your workflow. This is useful for discovering available tools from one or more servers and understanding their input schemas before integration with workflows.

First let's list the tools available from the `mcp_time` server. We should observe the following:

```
convert_time
get_current_time
```

In [ ]:
# For stdio transport
!nat mcp client tool list --transport stdio --command "python" --args "-m mcp_server_time"

Next let's list the tools available from the `mcp_math` server. We should observe the following:

```
calculator__subtract
calculator__add
current_datetime
calculator__compare
calculator__divide
calculator__multiply
react_agent
````

Note that the `react_agent` tool will be present in this list because it is a default chat capability that inherently gets deployed with the server.

In [ ]:
# For streamable-http transport (default)
!nat mcp client tool list --url http://localhost:9901/mcp

<a id="mcp-client-tools"></a>
## 2.3) Inspecting MCP client integration in workflows

Once an MCP server has been integrated with a workflow, it is often useful to inspect the tools integrated from the client side to confirm that tool discovery worked as expected.

When you serve a workflow that includes an `mcp_client` function group, the NeMo Agent toolkit exposes an HTTP endpoint to inspect the tools configured on the client side and whether each tool is available on the connected server. We can inspect this workflow and as follows:

In [ ]:
# let's kill anything using port 9901 in case it's already running
!lsof -ti:9901 | xargs kill -9

In [ ]:
%%bash --bg
nat mcp serve --config_file simple_calculator_notebook/configs/config.yml \
  --host 0.0.0.0 \
  --port 9901 \
  --name "My MCP Server"

In [ ]:
# Wait about 10 seconds before running the next cell...
import time

time.sleep(10)

<span style="color: red"><i>note: If running this notebook in a cloud provider such as Google Colab, `dask` may be installed. If it is, you will first have to uninstall it via:</i></span>

In [ ]:
#!uv pip uninstall dask

In [ ]:
# let's kill anything using port 8000 in case it's already running
!lsof -ti:8000 | xargs kill -9

In [ ]:
%%bash --bg
nat serve --config_file ./mcp_dev_workflow/configs/mcp_config.yml

In [ ]:
# Wait about 10 seconds before running the next cell...
import time

time.sleep(10)

In [ ]:
# 2. Verify the workflow server is running
!ps aux | grep "nat serve"

In [ ]:
# 3. Call the endpoint and pretty-print the response
!curl -s -v http://localhost:8000/mcp/client/tool/list | jq

Example output from above:

```
{
  "mcp_clients": [
    {
      "function_group": "mcp_time",
      "server": "stdio:python",
      "transport": "stdio",
      "session_healthy": true,
      "protected": false,
      "tools": [
        {
          "name": "convert_time",
          "description": "Convert time between timezones",
          "server": "stdio:python",
          "available": true
        },
        {
          "name": "get_current_time",
          "description": "Get current time in a specific timezones",
          "server": "stdio:python",
          "available": true
        }
      ],
      "total_tools": 2,
      "available_tools": 2
    },
    {
      "function_group": "mcp_math",
      "server": "streamable-http:http://localhost:9901/mcp",
      "transport": "streamable-http",
      "session_healthy": true,
      "protected": false,
      "tools": [
        {
          "name": "calculator__add",
          "description": "Add two or more numbers together.",
          "server": "streamable-http:http://localhost:9901/mcp",
          "available": true
        },
        {
          "name": "calculator__compare",
          "description": "Compare two numbers.",
          "server": "streamable-http:http://localhost:9901/mcp",
          "available": true
        },
        {
          "name": "calculator__divide",
          "description": "Divide one number by another.",
          "server": "streamable-http:http://localhost:9901/mcp",
          "available": true
        },
        {
          "name": "calculator__multiply",
          "description": "Multiply two or more numbers together.",
          "server": "streamable-http:http://localhost:9901/mcp",
          "available": true
        },
        {
          "name": "calculator__subtract",
          "description": "Subtract one number from another.",
          "server": "streamable-http:http://localhost:9901/mcp",
          "available": true
        },
        {
          "name": "current_datetime",
          "description": "Returns the current date and time in human readable format with timezone information.",
          "server": "streamable-http:http://localhost:9901/mcp",
          "available": true
        },
        {
          "name": "react_agent",
          "description": "ReAct Agent Workflow",
          "server": "streamable-http:http://localhost:9901/mcp",
          "available": true
        }
      ],
      "total_tools": 7,
      "available_tools": 7
    }
  ]
}
```

<a id="client-tools"></a>
## 2.4) Calling individual MCP tools

MCP tools can also be called individually using the `nat mcp client tool call` utility. This is very helpful for individual tool calls because it manages the MCP handshake and request simultaneously.

In [ ]:
%%bash
# Pass arguments as JSON
nat mcp client tool call calculator__multiply \
  --url http://localhost:9901/mcp \
  --json-args '{"numbers": [2, 3]}'

This section showed how to consume MCP servers by using the `nat` command line utility to deploy and manage MCP clients. With simple support for multiple transport types.

NeMo Agent toolkit also supports [MCP Authentication](../../docs/source/workflows/mcp/mcp-auth.md).

To use a protected MCP server, you need to provide the `--auth` flag:
```bash
nat mcp client tool list --url http://example.com/mcp --auth
```
This will use the `mcp_oauth2` authentication provider to authenticate the user. For more information, refer to [MCP Authentication](./mcp-auth.md).


<a id="next-steps"></a>
# 3.0 Next Steps

MCP servers follow the MCP protocol, and therefore can be used by any MCP client, not only NeMo Agent toolkit MCP clients. As a follow-up exercise, try connecting an MCP server to your Agentic IDE like VS Code GitHub co-pilot, Cursor, Windsurf or Claude Code.

Continue with the notebook series with `multi_agent_orchestration.ipynb`. Once agents are equipped with the tools they need to properly address user queries, we find a need to draw the line between one agent and another, and the concept of multi-agent systems and multi-agent orchestration takes center stage. In the next notebook, we will show how easily multi-agent orchestration can be coordinated using NeMo Agent toolkit and also show a quick demo of multi-modal asset handling with agents.